# Creating and Valuing a CDS Index Option

This notebook does the valuation of an option on a CDS index using Anderson's method and an Adjusted Black Method. For details see Modelling Singlename and Multiname Credit Derivatives by D.O'Kane.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 21:20 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [3]:
value_dt = Date(1, 8, 2007)
step_in_dt = value_dt.add_weekdays(0)
value_dt = step_in_dt
settle_dt = step_in_dt

## Build Libor Curve

In [4]:
dc_type = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(settle_dt, "1D", 0.0500, dc_type); depos.append(depo)

fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap_type = SwapTypes.PAY
swap1 = IborSwap(settle_dt,"1Y",swap_type,0.0502,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"2Y",swap_type,0.0502,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"3Y",swap_type,0.0501,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"4Y",swap_type,0.0502,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"5Y",swap_type,0.0501,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

In [5]:
cdsMaturity3Y = value_dt.next_cds_date(36)
cdsMaturity5Y = value_dt.next_cds_date(60)
cdsMaturity7Y = value_dt.next_cds_date(84)
cdsMaturity10Y = value_dt.next_cds_date(120)

## Load the Underlying CDS Index Portfolio

In [6]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
issuer_curves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    creditName = splitRow[0]
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])

    cds3Y = CDS(step_in_dt, cdsMaturity3Y, spd3Y)
    cds5Y = CDS(step_in_dt, cdsMaturity5Y, spd5Y)
    cds7Y = CDS(step_in_dt, cdsMaturity7Y, spd7Y)
    cds10Y = CDS(step_in_dt, cdsMaturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]

    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    issuer_curves.append(issuer_curve)

## Term Structure of CDS Index Market

We now set up the term structure of CDS Index trades and start by setting up the maturity dates of the index

In [7]:
indexMaturity3Y = Date(20,12,2009)
indexMaturity5Y = Date(20,12,2011)
indexMaturity7Y = Date(20,12,2013)
indexMaturity10Y = Date(20,12,2016)

Now we set their upfronts and cpns

In [8]:
index_upfronts = [0.0, 0.0, 0.0, 0.0]
index_maturity_dts = [indexMaturity3Y, indexMaturity5Y, indexMaturity7Y, indexMaturity10Y]
index_recovery = 0.40

## Setting up the Index Option

In [9]:
expiry_dt = Date(1,2,2008)
maturity_dt = Date(20,12,2011)
notional = 10000
volatility = 0.50
index_cpn = 0.004
tolerance = 1e-6

## Comparison of Anderson versus Adjusted Black Model

In [10]:
print("STRIKE    INDEX      PAY    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC")

for index in np.linspace(20, 50, 5):
    
    # Create a flat CDS index curve
    cds_contracts = []
    for dt in index_maturity_dts:
        cds = CDS(value_dt, dt, index / 10000.0)
        cds_contracts.append(cds)

    # Build the flat CDS index curve
    index_curve = CDSCurve(value_dt, cds_contracts, libor_curve, index_recovery)

    # Now we need to set up the underlying curves and adjust them
    indexSpreads = [index / 10000.0] * 4

    indexPortfolio = CDSIndexPortfolio()
    adjustedIssuerCurves = indexPortfolio.hazard_rate_adjust_intrinsic(value_dt, 
                                                                    issuer_curves, 
                                                                    indexSpreads, 
                                                                    index_upfronts,
                                                                    index_maturity_dts, 
                                                                    index_recovery, 
                                                                    tolerance)        
        
    for strike in np.linspace(20, 60, 5):    

        option = CDSIndexOption(expiry_dt,
                                   maturity_dt, 
                                   index_cpn, 
                                   strike / 10000.0, 
                                   notional)
        
        v_pay_1, v_rec_1, strike_value, mu, expH = option.value_anderson(value_dt, 
                                                                       adjustedIssuerCurves, 
                                                                       index_recovery,
                                                                       volatility)

        v_pay_2, v_rec_2 = option.value_adjusted_black(value_dt,
                                                     index_curve, 
                                                     index_recovery, 
                                                     libor_curve,
                                                     volatility)
            
        print("%6.3f   %6.3f   %8.3f   %8.3f   %8.3f   %8.2f   %8.2f   %8.2f   %8.2f" % 
              (strike, index, v_pay_1, v_rec_1, strike_value, mu, expH, v_pay_2, v_rec_2 ))

STRIKE    INDEX      PAY    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC


20.000   20.000     16.090      6.237    -70.799      22.87     -60.70      16.12       6.15
30.000   20.000      4.040     28.882    -35.287      22.85     -60.76       4.04      28.73
40.000   20.000      0.885     60.202      0.000      22.83     -60.82       0.90      59.98
50.000   20.000      0.188     93.764     35.064      22.82     -60.88       0.20      93.40
60.000   20.000      0.043    127.661     69.907      22.80     -60.94       0.05     127.08


20.000   27.500     41.049      1.478    -70.799      31.48     -30.23      41.09       1.40
30.000   27.500     17.502     12.604    -35.287      31.46     -30.27      17.50      12.50
40.000   27.500      6.329     35.885      0.000      31.45     -30.30       6.36      35.77
50.000   27.500      2.129     65.921     35.064      31.44     -30.34       2.16      65.71
60.000   27.500      0.681     98.493     69.907      31.43     -30.38       0.73      98.13


20.000   35.000     69.457      0.326    -70.799      40.09       0.08      69.55       0.30
30.000   35.000     39.405      4.926    -35.287      40.09       0.06      39.41       4.86
40.000   35.000     19.329     19.281      0.000      40.09       0.05      19.37      19.26
50.000   35.000      8.600     42.767     35.064      40.08       0.03       8.74      42.79
60.000   35.000      3.710     71.874     69.907      40.08       0.02       3.78      71.71


20.000   42.500     98.611      0.077    -70.799      48.72      30.23      98.72       0.07
30.000   42.500     65.726      1.821    -35.287      48.72      30.23      65.72       1.80
40.000   42.500     39.166      9.672      0.000      48.73      30.24      39.16       9.69
50.000   42.500     21.398     26.096     35.064      48.73      30.25      21.45      26.16
60.000   42.500     11.006     49.680     69.907      48.73      30.25      11.15      49.76


20.000   50.000    127.829      0.018    -70.799      57.37      60.24     127.92       0.01
30.000   50.000     93.877      0.674    -35.287      57.38      60.27      93.80       0.65
40.000   50.000     63.489      4.675      0.000      57.39      60.30      63.37       4.69
50.000   50.000     39.749     15.105     35.064      57.39      60.33      39.77      15.29
60.000   50.000     23.589     32.900     69.907      57.40      60.36      23.65      33.10


In [11]:
print(option)

OBJECT TYPE: CDSIndexOption
EXPIRY DATE: 01-FEB-2008
MATURITY DATE: 20-DEC-2011
INDEX cpn: 40.0bp
NOTIONAL: 10000
LONG PROTECTION: True
FREQUENCY: FrequencyTypes.QUARTERLY
DAYCOUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD



Copyright (c) 2020 Dominic O'Kane